In [1]:
from googleapiclient.discovery import build
import pandas as pd

In [2]:
# Setting API key

api_key = "YOUR API KEY"

api_service_name = "youtube"
api_version = "v3"

# Initializing Youtube data API client

youtube = build(api_service_name, api_version, developerKey = api_key)


In [3]:
# Channel ID

channel_id = "UC2J_VKrAzOEJuQvFFtj3KUw"  # CSK

In [4]:
# Function to get channel details

def get_channel_details(channel_id):
    
    request = youtube.channels().list(
        part="snippet, contentDetails, statistics",
        id=channel_id)
    response = request.execute()

    data = {
        "channel_id": channel_id,
        "channel_name": response['items'][0]['snippet']['title'],
        "channel_description": response['items'][0]['snippet']['description'],
        "channel_subscriber_count": int(response['items'][0]['statistics']['subscriberCount']),
        "channel_view_count": int(response['items'][0]['statistics']['viewCount']),
        "channel_video_count": int(response['items'][0]['statistics']['videoCount']),
        "Playlist_Id": response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    }
    
    return data

get_channel_details(channel_id)

channel_data = get_channel_details(channel_id)


# Function to get video IDs

def get_video_ids(playlist_id, max_results=10):
    video_ids = []
    next_page_token = None

    while len(video_ids) < max_results:
        request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=min(50, max_results - len(video_ids)),
            pageToken=next_page_token)
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return video_ids

playlist_id = (channel_data['Playlist_Id'])

max_results = 10  # Change this to the desired number of video IDs you want
video_ids = get_video_ids(playlist_id, max_results)

# Function to get comment details (For a single video - 5 comments)

def get_comment_data(video_id):
    comment_info = {}
        
    request = youtube.commentThreads().list(
                        part='snippet',
                        videoId=video_id, 
                        textFormat='plainText',
                        maxResults=5)
    response = request.execute()

    for index, comment in enumerate(response['items'],start=1):
        comment_id = comment['snippet']['topLevelComment']['id']
        comment_text = comment['snippet']['topLevelComment']['snippet']['textDisplay']
        comment_data = {
                 "Comment_Id": comment_id,
                 "Comment_Text": comment_text,
                 "Comment_Author": comment['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                 "Comment_PublishedAt": comment['snippet']['topLevelComment']['snippet']['publishedAt']}
        comment_info[f"Comment_Id_{index}"] = comment_data

    return comment_info

for i in range(len(video_ids)):
    video_id = video_ids[i]

get_comment_data(video_id)

comment_data = get_comment_data(video_id)


# Function to get video details

def get_video_data(video_ids):
    
    dict_video_data = {}
    
    for i in range(len(video_ids)):
        video_id = video_ids[i]
        
        request = youtube.videos().list(
                            part="snippet, statistics,contentDetails",
                            id = video_id)
        response = request.execute()
        
        video_data = {
            "Video_Id": video_id,
            "Video_Name": response['items'][0]['snippet']['title'],
            "Video_Description": response['items'][0]['snippet']['description'],
            "Video_View_Count": int(response['items'][0]['statistics'].get('viewCount',0)),
            "Video_Likes": int(response['items'][0]['statistics'].get('likeCount',0)),
            "Video_Dislikes": int(response['items'][0]['statistics'].get('dislikeCount',0)),
            "Video_PublishedAt": response['items'][0]['snippet']['publishedAt'],
            "Video_Comments_Count": int(response['items'][0]['statistics'].get('commentCount',0)),
        }
        if video_data['Video_Comments_Count'] > 0 :
            video_data['Video_Comments'] = comment_data
        else:
            video_data['Video_Comments'] = "No comments on this video"
            
            
        dict_video_data[f"Video_Id_{i+1}"] = video_data  
        
    return dict_video_data

get_video_data(video_ids)

video_data = get_video_data(video_ids)



In [5]:
channel_data

{'channel_id': 'UC2J_VKrAzOEJuQvFFtj3KUw',
 'channel_name': 'Chennai Super Kings',
 'channel_description': 'The Official YouTube Channel of the Chennai Super Kings. #WhistlePodu #Yellove',
 'channel_subscriber_count': 3140000,
 'channel_view_count': 622224892,
 'channel_video_count': 1097,
 'Playlist_Id': 'UU2J_VKrAzOEJuQvFFtj3KUw'}

In [6]:
video_data

{'Video_Id_1': {'Video_Id': 'Erkjh_RM5y0',
  'Video_Name': 'That moment when a billion hearts sang in unison! 🇮🇳💛#IndependenceDay 📹: @icc',
  'Video_Description': '',
  'Video_View_Count': 18785,
  'Video_Likes': 3151,
  'Video_Dislikes': 0,
  'Video_PublishedAt': '2023-08-15T13:22:02Z',
  'Video_Comments_Count': 25,
  'Video_Comments': {'Comment_Id_1': {'Comment_Id': 'UgyAwPuUQAug3hnQYbB4AaABAg',
    'Comment_Text': 'https://youtu.be/Ubi7tKRoiE4',
    'Comment_Author': 'Abhishek Rajput',
    'Comment_PublishedAt': '2023-08-17T09:57:36Z'},
   'Comment_Id_2': {'Comment_Id': 'UgyJGwZkMx_h5ncdTcZ4AaABAg',
    'Comment_Text': 'Ms 8no.😂',
    'Comment_Author': 'BG shorts',
    'Comment_PublishedAt': '2023-08-16T10:51:53Z'},
   'Comment_Id_3': {'Comment_Id': 'UgxkAIfyLGNtUR8Nb1x4AaABAg',
    'Comment_Text': 'My all time CSK Playing XI\n1. Shane Watson\n2.Faf du Plessis\n3.Ruturaj Gaikwad\n4.Suresh Raina (Mr.IPL) \n5.Ambati Rayudu\n6.MS Dhoni (Thala) \n7. Ravindra Jadeja (Jaddu) \n8.Dwanye Br

In [7]:
combined_data = {**channel_data, **video_data}

combined_data

{'channel_id': 'UC2J_VKrAzOEJuQvFFtj3KUw',
 'channel_name': 'Chennai Super Kings',
 'channel_description': 'The Official YouTube Channel of the Chennai Super Kings. #WhistlePodu #Yellove',
 'channel_subscriber_count': 3140000,
 'channel_view_count': 622224892,
 'channel_video_count': 1097,
 'Playlist_Id': 'UU2J_VKrAzOEJuQvFFtj3KUw',
 'Video_Id_1': {'Video_Id': 'Erkjh_RM5y0',
  'Video_Name': 'That moment when a billion hearts sang in unison! 🇮🇳💛#IndependenceDay 📹: @icc',
  'Video_Description': '',
  'Video_View_Count': 18785,
  'Video_Likes': 3151,
  'Video_Dislikes': 0,
  'Video_PublishedAt': '2023-08-15T13:22:02Z',
  'Video_Comments_Count': 25,
  'Video_Comments': {'Comment_Id_1': {'Comment_Id': 'UgyAwPuUQAug3hnQYbB4AaABAg',
    'Comment_Text': 'https://youtu.be/Ubi7tKRoiE4',
    'Comment_Author': 'Abhishek Rajput',
    'Comment_PublishedAt': '2023-08-17T09:57:36Z'},
   'Comment_Id_2': {'Comment_Id': 'UgyJGwZkMx_h5ncdTcZ4AaABAg',
    'Comment_Text': 'Ms 8no.😂',
    'Comment_Author': '